In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(1460, 81)
(1459, 80)


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window




In [0]:

trainfile = r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/train.csv'
trainData = pd.read_csv(trainfile)  #creates a dataframe
testfile = r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe
print(trainData.dtypes)
print(testData.shape)

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In [0]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
y_train = trainData["SalePrice"]#make copy of target column
x_train = trainData.drop(["SalePrice"], axis=1) #extracting training data without the target column

print(x_train.shape)

(1460, 80)


In [0]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])

combined_Data.isnull().sum()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


1stFlrSF            0
2ndFlrSF            0
3SsnPorch           0
Alley            2721
BedroomAbvGr        0
BldgType            0
BsmtCond           82
BsmtExposure       82
BsmtFinSF1          1
BsmtFinSF2          1
BsmtFinType1       79
BsmtFinType2       80
BsmtFullBath        2
BsmtHalfBath        2
BsmtQual           81
BsmtUnfSF           1
CentralAir          0
Condition1          0
Condition2          0
Electrical          1
EnclosedPorch       0
ExterCond           0
ExterQual           0
Exterior1st         1
Exterior2nd         1
Fence            2348
FireplaceQu      1420
Fireplaces          0
Foundation          0
FullBath            0
Functional          2
GarageArea          1
GarageCars          1
GarageCond        159
GarageFinish      159
GarageQual        159
GarageType        157
GarageYrBlt       159
GrLivArea           0
HalfBath            0
Heating             0
HeatingQC           0
HouseStyle          0
Id                  0
KitchenAbvGr        0
KitchenQua

In [0]:
#Define threshold for dropping columns
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

1751
(2919, 75)


In [0]:
#look at what other columns have missing values
td1.isnull().sum()

1stFlrSF           0
2ndFlrSF           0
3SsnPorch          0
BedroomAbvGr       0
BldgType           0
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       2
BsmtHalfBath       2
BsmtQual          81
BsmtUnfSF          1
CentralAir         0
Condition1         0
Condition2         0
Electrical         1
EnclosedPorch      0
ExterCond          0
ExterQual          0
Exterior1st        1
Exterior2nd        1
Fireplaces         0
Foundation         0
FullBath           0
Functional         2
GarageArea         1
GarageCars         1
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt      159
GrLivArea          0
HalfBath           0
Heating            0
HeatingQC          0
HouseStyle         0
Id                 0
KitchenAbvGr       0
KitchenQual        1
LandContour        0
LandSlope          0
LotArea            0
LotConfig          0
LotFrontage  

In [0]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())
  
print(td1.head(20))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


      1stFlrSF  2ndFlrSF  3SsnPorch  BedroomAbvGr BldgType BsmtCond BsmtExposure  BsmtFinSF1  BsmtFinSF2 BsmtFinType1 BsmtFinType2  BsmtFullBath  BsmtHalfBath BsmtQual  BsmtUnfSF CentralAir Condition1 Condition2 Electrical  EnclosedPorch ExterCond ExterQual Exterior1st Exterior2nd  Fireplaces Foundation  FullBath Functional  GarageArea  GarageCars GarageCond GarageFinish GarageQual GarageType  GarageYrBlt  GrLivArea  HalfBath Heating HeatingQC HouseStyle  Id  KitchenAbvGr KitchenQual LandContour LandSlope  LotArea LotConfig  LotFrontage LotShape  LowQualFinSF  MSSubClass MSZoning  MasVnrArea MasVnrType  MiscVal  MoSold Neighborhood  OpenPorchSF  OverallCond  OverallQual PavedDrive  PoolArea RoofMatl RoofStyle SaleCondition SaleType  ScreenPorch Street  TotRmsAbvGrd  TotalBsmtSF Utilities  WoodDeckSF  YearBuilt  YearRemodAdd  YrSold
0 0        856       854          0             3     1Fam       TA           No       706.0         0.0          GLQ          Unf           1.0           0

In [0]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES 
td1.isnull().sum()

1stFlrSF           0
2ndFlrSF           0
3SsnPorch          0
BedroomAbvGr       0
BldgType           0
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         0
BsmtFinSF2         0
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       0
BsmtHalfBath       0
BsmtQual          81
BsmtUnfSF          0
CentralAir         0
Condition1         0
Condition2         0
Electrical         1
EnclosedPorch      0
ExterCond          0
ExterQual          0
Exterior1st        1
Exterior2nd        1
Fireplaces         0
Foundation         0
FullBath           0
Functional         2
GarageArea         0
GarageCars         0
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt        0
GrLivArea          0
HalfBath           0
Heating            0
HeatingQC          0
HouseStyle         0
Id                 0
KitchenAbvGr       0
KitchenQual        1
LandContour        0
LandSlope          0
LotArea            0
LotConfig          0
LotFrontage  

In [0]:
#DROP COLUMNS THAT STILL HAVE NULL VALUES
print(td1.shape)
#td1=td1.drop(columns=["PropertyField32", "PropertyField34"])
#td1=td1.drop(columns=["PropertyField5"])
td1.isnull().sum()
print(td1.shape)

(2919, 75)
(2919, 75)


In [0]:
#DO ONE-HOT ENCODING ON CATEGORICAL VARIABLES==============================================
#The below function returns a list of categorical features which are not numeric. 
train_cat_cols = td1.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)

#If there are categorical columns which are encoded as numeric ones 
#then we need to explicitly enter the column names in a list and concatenate the two lists in python.
#ONE-HOT ENCODING-generate one-hot encoding on a common basis -THIS TAKES 30 MINS

combined_Data = pd.get_dummies(td1,train_cat_cols)
combined_Data.shape


(38,)
Index(['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Foundation', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSZoning', 'MasVnrType', 'Neighborhood', 'PavedDrive', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'Street', 'Utilities'], dtype='object')


(2919, 271)

In [0]:
#Separate Train data and test data
trainData = combined_Data.xs(0)
testData = combined_Data.xs(1)
print(trainData.shape)
print(testData.shape)

trainData=pd.concat([trainData,y_train], axis=1)
print(trainData.shape)


(1460, 271)
(1459, 271)
(1460, 272)


In [0]:
export_csv = trainData.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/Preprocess_Train.csv')
exporttest_csv = testData.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/Preprocess_Test.csv')